<a href="https://colab.research.google.com/github/40love/licenta2019/blob/master/ColabPrj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fisier Antrenament

In [0]:
!pip install pycrypto

In [0]:
#run once to import to files your google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
'''MAININPUT
Created on Apr 24, 2019

@author: PITA
'''
#from Package import GetInput, Retea 
import numpy
limita = 0.5
def tipareste(x):
    lst=[]
    for it in range(len(x)):
        for jt in range(len(x[it])):
          if x[it][jt] not in lst:
              lst.append(x[it][jt])
    lst.sort(reverse=True)
    print(lst)  
    return lst
def analizeazaPoza(poza):
    import cv2
    pixeli = GetInput.prelucreazaImagineInputAux(poza)
    image = cv2.imread(poza)
    model = Retea.getModel()
    pred = model.predict(numpy.array([pixeli]))
    suma = 0
    cnt = 0
    for it in range(len(pred[0])):
        
        for jt in range(len(pred[0][it])):
            
            if pred[0][it][jt]>limita:
                suma +=pred[0][it][jt]
                cnt +=1
    medie = (float(suma)/float(cnt))
    for it in range(len(pred[0])):
        for jt in range(len(pred[0][it])):
            if pred[0][it][jt]>medie:    
                cv2.circle(image,(it,jt), 3, (0,0,255), -1)
    #tipareste(pred)
    cv2.imshow("IataCarii", image)
    cv2.waitKey(0)
    print(medie)
    print(pred.shape)
    print(pred)
    
    
def analizeazaPozaDebug(poza):
    import cv2
    from keras.models import model_from_json
    #model = Antrenament.incarcaModel()
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    model = loaded_model
    pixeli = GetInput.prelucreazaImagineInputAux(poza)
    image = cv2.imread(poza)
    
    pred = model.predict(numpy.array([pixeli]))
    suma = 0
    cnt = 0
    for it in range(len(pred[0])):
        
        for jt in range(len(pred[0][it])):
            
            if pred[0][it][jt]>limita:
                suma +=pred[0][it][jt]
                cnt +=1
    medie = (float(suma)/float(cnt))
    #medie = 5000000
    for it in range(len(pred[0])):
        for jt in range(len(pred[0][it])):
            if pred[0][it][jt]>medie:    
                #cv2.circle(image,(it*4,jt*4), 3, (0,0,255), -1)
                pass
    lst = tipareste(pred[0])
    for kt in lst :
        for it in range(len(pred[0])):
            for jt in range(len(pred[0][it])):
                if pred[0][it][jt]==kt:    
                    cv2.circle(image,(it,jt), 3, (0,0,255), -1)
                    cv2.imshow("IataCarii", image)
                    cv2.waitKey(0)
                    
    #cv2.imshow("IataCarii", image)
    cv2.waitKey(0)
    print(medie)
    print(pred.shape)
    print(pred)
    
'''GetInput
Created on Apr 24, 2019
@author: PITA
'''
from PIL import Image
import numpy
import os
from builtins import isinstance

directorImaginiInput="/content/drive/My Drive/Colab Notebooks/licenta2019/Radiografii_DB"
directorImaginiOutput="/content/drive/My Drive/Colab Notebooks/licenta2019/Radiografii_DB_Output"
dimensiune = (256,256)
dimensiuneOut = (256,256)
factorZoom = dimensiune[0]/dimensiuneOut[0]
def tranformaImagine(imagine):
    imagine=Image.open(imagine).convert('L')
    #image = color.rgb2gray(imagine)
    image_resized = imagine.resize(dimensiune,Image.ANTIALIAS)
    pixels = list(image_resized.getdata())
    width, height = image_resized.size
    pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
    return pixels

def getPixeliOutput(fisier):
    output = []
    # import the necessary packages
    import imutils
    import cv2
    

    # load the image, convert it to grayscale, blur it slightly,
    # and threshold it
    image = cv2.imread(fisier)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = numpy.array([0,100,100])
    upper_red = numpy.array([179, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    #cv2.imshow("HSV", hsv)
   # cv2.waitKey(0)
   # cv2.imshow("Mask", mask)
   # cv2.waitKey(0)
    
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
    # find contours in the thresholded image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # Create a black image
    img = numpy.zeros((600,600,3), numpy.uint8)
    for c in cnts:
        # compute the center of the contour
        M = cv2.moments(c)
        print(M["m10"])
        print(M["m00"])
        print(M["m01"])
        if M["m00"] != 0.0 :
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])   
            output.append([cX,cY])
            cv2.circle(img,(cX,cY), 3, (0,0,255), -1)
    
    print(fisier)  
    print(output)
    #cv2.imshow("Mask", img)
    #cv2.waitKey(0)
    return output

def prelucreazaImagineOutputAux(fisier):
    numeComplet = os.path.join(directorImaginiOutput,fisier)
    pixeliOutput = getPixeliOutput(numeComplet)
    cheieDictionar = fisier.split(".")[0]
    if cheieDictionar not in dictionarImagini:
        dictionarImagini[cheieDictionar]=[0,0]
    y_t = []
    y = [0]*dimensiuneOut[0]
    for _ in range(dimensiuneOut[1]):
        y_t.append(y)
    for pixel in pixeliOutput:
        for kt in range(int(pixel[0]/factorZoom)-1,int(pixel[0]/factorZoom)+2):
            for lt in range(int(pixel[1]/factorZoom)-1,int(pixel[1]/factorZoom)+2):
                y_t[kt][lt] = 1
        #y_t[int(pixel[0]/factorZoom)][int(pixel[1]/factorZoom)] = 1
    y_t = numpy.reshape(y_t,(dimensiune[0],dimensiune[1],1))
    dictionarImagini[cheieDictionar][1]=y_t
    return y_t

def prelucreazaImagineOutput(fisier):
    fisierMirror = mirrorImage(fisier,directorImaginiOutput)
    
    prelucreazaImagineOutputAux(fisier)
    prelucreazaImagineOutputAux(fisierMirror)
    
def prelucreazaImagineInputAux(fisier):
    numeComplet = os.path.join(directorImaginiInput,fisier)
    img = tranformaImagine(numeComplet)
    img=numpy.reshape(img,(dimensiune[0],dimensiune[1],1))
    if fisier.split(".")[0] not in dictionarImagini:
        dictionarImagini[fisier.split(".")[0]] = [0,0]
    
    dictionarImagini[fisier.split(".")[0]][0]=img
    return img
    
    
def mirrorImage(fisier,director):
    numeComplet = os.path.join(director,fisier)
    image_obj = Image.open(numeComplet)
    rotated_image = image_obj.transpose(Image.FLIP_LEFT_RIGHT)
    fisierMirror = fisier.split(".")[0]+"mirror."+fisier.split(".")[1]
    numeCompletMirror = os.path.join(director,fisierMirror)
    rotated_image.save(numeCompletMirror)
    return fisierMirror

def prelucreazaImagineInput(fisier):
    fisierMirror = mirrorImage(fisier,directorImaginiInput)
    prelucreazaImagineInputAux(fisier)
    prelucreazaImagineInputAux(fisierMirror)

dictionarImagini={}  #dictionar[NumeImagine] = [intrarePiexeli,iesirePixelCarii]

def resizePoza(directorImagini,fisier):
    numeComplet = os.path.join(directorImagini,fisier)
    imagine=Image.open(numeComplet)#.convert('L')
    image_resized = imagine.resize(dimensiune,Image.ANTIALIAS)
    numeOutput = fisier.split(".")[0] + "aux"+"."+fisier.split(".")[1]
    numeComplet = os.path.join(directorImagini,numeOutput)
    image_resized.save(numeComplet)
    return numeOutput

def mainInput():
    from os import listdir
    from os.path import isfile, join
    onlyfiles = [f for f in listdir(directorImaginiInput) if isfile(join(directorImaginiInput, f))]
    for fisier in onlyfiles:
        if (fisier.endswith("jpg") or fisier.endswith("png") ) and "aux" not in fisier:
            fisier = resizePoza(directorImaginiInput,fisier)
            prelucreazaImagineInput(fisier)
            
    onlyfiles = [f for f in listdir(directorImaginiOutput) if isfile(join(directorImaginiOutput, f))]
    for fisier in onlyfiles:
        if (fisier.endswith("jpg") or fisier.endswith("png") ) and "aux" not in fisier:
            fisier = resizePoza(directorImaginiOutput,fisier)
            prelucreazaImagineOutput(fisier)
            
    intrare = []
    iesire = []
    for valori in dictionarImagini.values():
        if(not isinstance(valori[0],int)) and (not isinstance(valori[1], int)):
            intrare.append(valori[0])
            iesire.append(valori[1])
    intrare = numpy.array(intrare)
    iesire =numpy.array(iesire)
    print(len(intrare), len(iesire))
    return intrare,iesire


'''Retea
Created on Apr 24, 2019

@author: PITA
'''
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Input,Flatten,Reshape,Dropout,BatchNormalization,Conv2DTranspose,UpSampling2D,concatenate
from keras import optimizers, losses
from keras import  regularizers
from keras.regularizers import l2
import keras.metrics
import keras.backend as K
import tensorflow as tf

dimensiune = (256,256)

def carenteTotale(y_true, y_pred):

    return K.sum(K.cast(K.equal(y_true,K.sign(y_pred)),tf.int16))


def pusaCarieSiNuE(y_true, y_pred):
    aux1 = K.not_equal(y_true,K.sign(y_pred))
    return K.sum(K.cast(K.equal(aux1,K.cast(K.sign(y_pred),tf.bool)),tf.int16))
    

def trebeCarieSiNaPus(y_true, y_pred):
    return carenteTotale(y_true, y_pred) + pusaCarieSiNuE(y_true, y_pred)
    '''
aux1 = K.not_equal(y_true,K.sign(y_pred))
aux2 =  K.not_equal(K.ones(K.shape(y_pred)),K.sign(y_pred))
return K.sum(K.cast(K.equal(aux1,aux2),tf.int16))
'''
def getModel():
    global model
    return model


from keras.optimizers import *
x=64

inputs = Input((dimensiune[0], dimensiune[1], 1))
conv1 = Conv2D(x, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = Conv2D(x, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(x*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(x*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(x*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(x*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(x*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(x*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(x*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(x*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(x*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = concatenate([drop4,up6], axis = 3)
conv6 = Conv2D(x*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(x*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(x*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(x*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(x*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(x*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(x*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(x*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(x, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(x, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(x, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(1, 1, activation = 'elu',name="myOutputLayer")(conv9)

model = Model(input = inputs, output = conv10)

model.compile(optimizer = Adam(lr = 1e-4), loss = keras.losses.poisson, metrics = [carenteTotale,pusaCarieSiNuE])

model.summary()

'''Antrenament
Created on May 2, 2019

@author: narci
'''
import imutils
import numpy
print(imutils.__version__)
#from Package.GetInput import mainInput
#from Package import Retea
#from Crypto.Random.random import shuffle
import tensorflow as tf
#from Package.MainScript import analizeazaPoza
from keras.models import model_from_json

def mainAntrenament() :
    #session = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    intrare, iesire  = mainInput()
    #intrare = numpy.tile(intrare,10)
    #iesire = numpy.tile(iesire, 10)
    model = getModel()
    
    model.fit (intrare,iesire,epochs=10,batch_size=10,shuffle=True,validation_split=0.1)
    salveazaModel(model)

#folder = r'C:\Users\PITA\git\licenta2019\Radiografii_DB'
def  deleteAuxFiles(folder):
    import os, shutil
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                if "aux" in file_path:
                    os.unlink(file_path)
            #elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
    print("Done")
    
   
   
def salveazaModel(model):     
        # serialize model to JSON
        model_json = model.to_json()
        with open("model.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights("model.h5")
        print("Saved model to disk")
 
def incarcaModel():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model
    

mainAntrenament()
analizeazaPoza(r"/content/drive/My Drive/Colab Notebooks/licenta2019/Radiografii_DB/PHOTO-2019-04-20-15-56-23.jpg")
deleteAuxFiles("/content/drive/My Drive/Colab Notebooks/licenta2019/Radiografii_DB") 
deleteAuxFiles("/content/drive/My Drive/Colab Notebooks/licenta2019/Radiografii_DB_Output") 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:399: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("my...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 256, 256, 64) 640         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, 256, 256, 64) 36928       conv2d_105[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_23 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_106[0][0]                 
__________________________________________________________________________________________________
conv2d_107